In [3]:
import os
import gnssrefl.gps as g
import gnssrefl.rinex2snr as rnx
import gnssrefl.quickLook_function as quick
import gnssrefl.gnssir as guts
import json
import pandas as pd 
import check_parameters
import requests
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(style="whitegrid");
import re
from datetime import datetime
import numpy as np

# making sure that env variables are set - if they are then nothing will print to screen
g.check_environ_variables()

%matplotlib inline

ModuleNotFoundError: No module named 'check_parameters'

### Marshall, Colorado, USA

**Station Name:** p041 

**Location:** Boulder, CO, USA

**Archive:** [UNAVCO](http://www.unavco.org)

**DOI:**  	[https://doi.org/10.7283/T5R49NQQ](https://doi.org/10.7283/T5R49NQQ)

**Ellipsoidal Coordinates:**

- Latitude: 39.94949

- Longitude: -105.19427

- Height: 1728.842 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/P041)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/P041.sta)

[Google Map Link](https://goo.gl/maps/GwGV8PS4CQVQzYHC7) 

<img src="https://gnss-reflections.org/static/images/P041.jpg" width="500">

## Data Summary

The p041 antenna is ~2 meters above the soil surface. It is located at Marshall Mesa, Colorado.
The site is relatively planar and free of obstructions. Since October 2018 the site has 
recorded multi-GNSS signals. Marshall Mesa has been featured in multiple publications on GNSS-IR:

* [Use of GPS Receivers as a Soil Moisture Network for Water Cycle Studies (2008)](https://www.kristinelarson.net/wp-content/uploads/2015/10/larson_soil_grl2008.pdf)

* [Can We Measure Snow Depth with GPS Receivers (2009)](https://www.kristinelarson.net/wp-content/uploads/2015/10/larsonetal_snow_2009.pdf) 

* [Sensing Vegetation Growth with Reflected GPS Signals (2010)](https://www.kristinelarson.net/wp-content/uploads/2015/10/small_etal_2010.pdf) 

To get a sense of whether an azimuth or elevation mask is appropriate, 
check the [Reflection Zone Mapping in the web app](https://gnss-reflections.org/rzones?station=p041&lat=39.9495&lon=-105.1943&height=1728.842&msl=on&RH=2&eang=2&azim1=0&azim2=360).  
In the linked page, the reflection zones from 5 to 25 degree elevation angles are plotted as 
colored ellipses surrounding the station.  

### Web App

More information on the station can be obtained from the GNSS-IR Web App, where p041 is one of the example cases for soil moisture and snow depth. [Please note that the app will be analyzing data in real-time, so please wait for the periodogram to appear on the left side of the page. It takes about 5 seconds](https://gnss-reflections.org/fancy6?example=p041). The web app will return a photograph, station coordinates, a Google Earth Map, and a sample periodogram. The periodogram plots the reflector height (RH) in four quadrants (NW, NE, SW, SE), allowing the reflection characteristics and quality of the site to be inferred. For example, variations in topography or buildings blocking the reflections could all affect the periodograms.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=p041" width="1000" height="900"></iframe>

### Setting Azimuth and Elevation Mask

To get a sense of whether an azimuth or elevation mask is appropriate, check the [Reflection Zone Mapping in the web app](https://gnss-reflections.org/rzones?station=p041&lat=39.9495&lon=-105.1943&height=1728.842&msl=on&RH=2&eang=2&azim1=0&azim2=360).  
In the linked page, the reflection zones at 5, 10, and 15-degree elevation angles are plotted as colored ellipses surrounding the station, all overlaid on a Google Earth map.  Higher elevation angles are closer to the station.
As can be seen from the Google Earth map, this site is relatively flat with no major obstacles to interfere with reflected signals.  Therefore the elevation angles can be left at default settings and no azimuth masks are required.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones?station=p041&lat=39.9495&lon=-105.1943&height=1728.842&msl=on&RH=2&eang=2&azim1=0&azim2=360" width="950" height="500"></iframe>

### Make SNR File

Begin by making an SNR file.

In [ ]:
station = 'p041'
year = 2020 
doy = 132

lat = 39.9495
long = -105.1943
height = 1728.856

In [ ]:
# Can view the parameters here
check_parameters.rinex2snr?

In [ ]:
args = check_parameters.rinex2snr(station,year,doy, translator='hybrid')
rnx.run_rinex2snr(**args)

### Take a Quick Look at the Data

Then use **quickLook** to analyze the reflection characteristics of the site [(For details on quickLook output)](../../docs/quickLook_desc.md).

The default return is for the L1 frequency:

In [ ]:
# making a plotting function for the quicklook function
def quicklook_results(args, values):
    freq = {1:'L1', 20: 'L2C', 5:'L5'}
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,10))
    quadrants = ['NW', 'NE', 'SW', 'SE']
    axes = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

    for i, quadrant in enumerate(quadrants):
        satellites = values[quadrant].keys()
        fail_satellites = values[f'f{quadrant}'].keys()

        for failsat in fail_satellites:
            axes[i].plot(values[f'f{quadrant}'][failsat][0], values[f'f{quadrant}'][failsat][1], color='lightgrey') 
        for sat in satellites:
            axes[i].plot(values[quadrant][sat][0], values[quadrant][sat][1])

    ax[0,0].set_title('Northwest', size=14)
    ax[0,1].set_title('Northeast',size=14)
    ax[1,0].set_title('Southwest', size=14)
    ax[1,1].set_title('Southeast', size=14)

    for ax in axes:
        ax.set_xlabel('reflector height (m)', size=14)
        ax.set_ylabel('volts/volts', size=14)
        ax.grid()
    
    fig.suptitle(f'GNSS Station {args["station"].upper()}, {args["year"]} doy {args["doy"]}, freq {freq[args["f"]]}, elevation angles {args["e1"]}-{args["e2"]} \n', size=16)
    fig.tight_layout()
    plt.show()
    
    
def quicklook_metrics(args, values):
#     fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
    quadrants = ['NW', 'NE', 'SW', 'SE']
    
    # re-organizing the data in a plotting friendly format
    success_data = {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise':[], 'Amplitude': []}
    fail_data =  {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise': [], 'Amplitude': []}
    
    for i, quadrant in enumerate(quadrants):
        for j in values[quadrant].keys():
            success_data['Azimuth'].append(datakeys[quadrant][j][0])
            success_data['Reflector Height'].append(datakeys[quadrant][j][1])
            success_data['Peak to Noise'].append(datakeys[quadrant][j][5])
            success_data['Amplitude'].append(datakeys[quadrant][j][4])
        for k in values[f'f{quadrant}'].keys():
            fail_data['Azimuth'].append(datakeys[f'f{quadrant}'][k][0])
            fail_data['Reflector Height'].append(datakeys[f'f{quadrant}'][k][1])
            fail_data['Peak to Noise'].append(datakeys[f'f{quadrant}'][k][5])
            fail_data['Amplitude'].append(datakeys[f'f{quadrant}'][k][4])

    return pd.DataFrame(success_data), pd.DataFrame(fail_data)   

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

Now try looking at the periodogram for L2C:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, f=20)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

Note that there are more colors in the L1 plots than in the L2C plots. That is the result of the fact that there are more L1 satellites than L2C satellites.

Now try L5:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, f=5)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

The L5 signal has only been available on satellites launched after 2010, so there are
fewer satellite tracks than either L1 or L2C.

The *quickLook* code has multiple options. For example, it is possible change the reflector height range:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, h1=0.5, h2=10)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

To look at Glonass and Galileo signals, the SNR files must be created using the -orb gnss flag.
If you have already made a file using only the GPS data, you will need the overwrite flag.

In [ ]:
# # need to remove old file otherwise, it will see the file exists and not create a new one with the new parameters
#os.remove('2020/snr/p041/p0411320.20.snr66')
args = check_parameters.rinex2snr(station, year, doy, orb='gnss', overwrite=True, translator='fortran')
rnx.run_rinex2snr(**args)

Beidou signals are tracked at this site, but unfortunately the data are not available in the RINEX 2.11 file.
They are very likely available in the RINEX 3 file, so you are encouraged to look there.

**quickLook** is meant to be a visual assessment of the spectral characteristics at a given site on a given day. For routine analysis, one must use  **gnssir**.

## Analyze the Data

We will start by setting up the analysis parameters. These are stored
in a json file. In this case, the p041 RINEX data are multi-gnss, so you could
set the options to allow all frequencies from all constellations by setting the parameter allfreq=True.

We are going to concentrate on GPS-only, which is the default. We have set stricter QC values by
setting the amplitude minimum to 8 and the peak 2 noise ratio to 3.2:

In [ ]:
# Here we can see what parameters are available and what the defaults are:
check_parameters.make_json?

In [ ]:
# we saved the lat, long, and height earlier
check_parameters.make_json(station, lat, long, height, peak2noise=3.2, ampl=8)

Because the site is uniformly flat, the parameters canbe left at default settings. The elevation angles for the SNR data are set to minimum and maximum values of 5 and 25 degrees, respectively.  The json output will be stored in $REFL_CODE/input/p041.json.

In [ ]:
# This is the json file that was created
json_file = 'input/p041.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)

os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

We are going to look at a subset of p041 data from 2019/2020 to look at changes due to
snow accumulation. The series will begin doy 245 (2019) and end on doy 70 (2020).

In [ ]:
#check_parameters.rinex2snr?

In [ ]:
args = check_parameters.rinex2snr(station, year=2019, doy=245, doy_end=365, archive='unavco', translator='hybrid')
rnx.run_rinex2snr(**args)

In [ ]:
args = check_parameters.rinex2snr(station, year=2020, doy=1, doy_end=70, archive='unavco', translator='hybrid')
rnx.run_rinex2snr(**args)

Now run gnssir for 2019/2020:

In [ ]:
#check_parameters.gnssir?

In [ ]:
year = 2019
doy = 1
doy_end = 366
year_end = 2020
pt=False
args = check_parameters.gnssir(station, year, doy, doy_end=doy_end, year_end=year_end, plt=pt, screenstats=False)
year_list = list(range(year, args['year_end'] + 1))
doy_list = list(range(doy, args['doy_end'] + 1))
for year in year_list:
    args['args']['year'] = year
    for doy in doy_list:
        args['args']['doy'] = doy
        guts.gnssir_guts(**args['args'])

Typically a daily average is sufficient for climatology studies.
To ensure the average is meaningful and not impacted by large outliers,
a median filter (meters) is used and a minimum number of tracks is required. Here a median filter of 0.15 meter is used and 80 tracks are required. Either of these parameters can be changed depending on your site.
In this particular example, I only used three GPS frequencies, L1, L2C, and L5.

In [ ]:
#check_parameters.daily_avg?

In [ ]:
check_parameters.daily_avg(station, medfilter=.15, ReqTracks=80, plt2screen=False, txtfile='p041-dailyavg.txt')

In [ ]:
import matplotlib.pyplot as plt

def read_allRH_file(filepath, regex):
    data = {'dates': [], 'rh': []}
    #read daily average reflector heights
    with open(f'{refl_code_dir}{filepath}', 'r') as myfile:
        file = myfile.read()
        matches = re.finditer(regex, file, flags=re.MULTILINE)

        for match in matches:
            ydoy = f'{int(match.group("year"))}-{int(match.group("doy"))}'
            date = datetime.strptime(ydoy, '%Y-%j').date()
            data['dates'].append(date)
            data['rh'].append(float(match.group('rh')))
            
    return data

regex = '^ (?P<year>[ \d]+) +(?P<doy>[\d]+) +(?P<rh>[\d|-|.]+)'
filepath = f'/Files/{station}_allRH.txt'
data = read_allRH_file(filepath, regex)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(10,8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim(2.1, 1.4)
g.set_ylabel('Reflector Height (m)');

A summary of how many values are being used in the daily average:

In [ ]:
plt.figure(figsize=(10,8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

And the daily RH average:

In [ ]:
regex = '^ (?P<year>[ \d]+) +(?P<doy>[\d]+) +(?P<rh>[\d|-|.]+)'
filepath = f'/Files/{station}-dailyavg.txt'
data = read_allRH_file(filepath, regex)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])

plt.figure(figsize=(10,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(2,1.7)
g.set_ylabel('Reflector Height (m)')
g.set_title(f'{station}: Daily Mean Reflector Height', size=16);